# Cleaning all the CSVs


In [1]:
# Setup: Create folder structure for local development
from pathlib import Path

# Create all necessary directories
data_dir = Path("./data")
raw_dir = data_dir / "raw"
clean_dir = data_dir / "clean"
final_dir = data_dir / "final"

# Create directories if they don't exist
for directory in [raw_dir, clean_dir, final_dir]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created directory: {directory}")

print(f"\n📁 Folder structure ready!")
print(f"📁 Put your CSV files in: {raw_dir.absolute()}")
print(f"📁 Cleaned files will go to: {clean_dir.absolute()}")  
print(f"📁 Final merged file will go to: {final_dir.absolute()}")

✓ Created directory: data\raw
✓ Created directory: data\clean
✓ Created directory: data\final

📁 Folder structure ready!
📁 Put your CSV files in: c:\Users\prana\OneDrive\Desktop\NASA-ASRS-Scraper\data\raw
📁 Cleaned files will go to: c:\Users\prana\OneDrive\Desktop\NASA-ASRS-Scraper\data\clean
📁 Final merged file will go to: c:\Users\prana\OneDrive\Desktop\NASA-ASRS-Scraper\data\final


In [2]:
import pandas as pd
from pathlib import Path

# --- paths ----------------------------------------------------------
# Updated for local Windows development (not Google Colab)
raw_dir   = Path("./data/raw")  # Put your CSV files here
clean_dir = Path("./data/clean")
big_csv_dir = Path("./data/final")

# Create directories if they don't exist (with parents=True for nested folders)
clean_dir.mkdir(parents=True, exist_ok=True)
big_csv_dir.mkdir(parents=True, exist_ok=True)

# --- settings -------------------------------------------------------
keep_cols = ["ACN", "Assessments", "Assessments.1", "Report 1", "Component"]

for csv_path in raw_dir.glob("*.csv"):          # every CSV in the folder
    # 1) read
    try:
        df = pd.read_csv(
            csv_path,
            engine="python",    # slower but more forgiving
            dtype=str,          # keep text intact
            on_bad_lines="skip" # skip any malformed rows
        )
    except pd.errors.ParserError as err:
        print(f"❌  {csv_path.name}: {err}")
        continue

    # 2) drop the first two rows and reset the index
    df = df.drop(df.index[:2]).reset_index(drop=True)

    # 3) rename the first (unnamed) column to "ACN"
    df.rename(columns={df.columns[0]: "ACN"}, inplace=True)

    # 4) drop rows that are missing ANY of the key fields
    df = df.dropna(subset=["Report 1"], how="any")
    #    (If you only want to drop when *all three* are empty, replace how="any"
    #     with: mask = (df["Assesments"].isna() & df["Assesments.1"].isna()) | df["Report 1"].isna()
    #     df = df[~mask])

    # 5) keep only the requested columns (skip any that aren't present)
    df = df[[c for c in keep_cols if c in df.columns]]

    # 6) write the cleaned file next to the originals
    df.to_csv(clean_dir / csv_path.name, index=False)

    print(f"✓ cleaned {csv_path.name}")

# Merging all the CSVs

In [ ]:
import pandas as pd
from pathlib import Path

clean_dir = Path("./data/clean")

# Check if clean directory exists and has CSV files
if not clean_dir.exists():
    print(f"❌ Directory {clean_dir} doesn't exist!")
    print("🔧 Run the setup cell (cell 2) first to create folders")
    print("🔧 Then run the cleaning cell (cell 3) to process your CSV files")
    exit()

csv_files = list(clean_dir.glob("*.csv"))
if len(csv_files) == 0:
    print(f"❌ No CSV files found in {clean_dir}")
    print("🔧 Make sure you:")
    print("   1. Put your raw CSV files in ./data/raw/")
    print("   2. Run the cleaning cell (cell 3) first")
    print(f"📁 Raw files should be in: {Path('./data/raw').absolute()}")
    exit()

# 1) build a list of DataFrames
dfs = [pd.read_csv(f, dtype=str)          # dtype=str → keep everything as text / avoids mixed-type warnings
       for f in csv_files]                # iterate through every cleaned file

# 2) concatenate row-wise
big_df = pd.concat(dfs, ignore_index=True)  # ignore_index=True → 0,1,2,… across all rows

print(f"✓ merged {len(dfs)} files from {clean_dir}")

❌ No CSV files found in data\clean
🔧 Make sure you:
   1. Put your raw CSV files in ./data/raw/
   2. Run the cleaning cell (cell 3) first
📁 Raw files should be in: c:\Users\prana\OneDrive\Desktop\NASA-ASRS-Scraper\data\raw


ValueError: No objects to concatenate

: 

In [ ]:
big_df

,ACN,Assessments,Assessments.1,Report 1,Component
0,177042,NaN,Aircraft,CABIN PRESSURE LOSS AT 35000'; RAPID DSNT MADE...,NaN
1,177186,Airport,Airport,JUST AFTER T/D ON RWY 27 AT Y31; A DEER RAN ON...,NaN
2,177189,NaN,Human Factors,I DEPARTED SOUTH LAKE TAHOE ARPT ON A DIRECT F...,NaN
3,177197,NaN,Human Factors,I TOOK OFF OUT OF VICTORIA ARPT AND WAS IN IMC...,NaN
4,177205,NaN,Human Factors,ON A FLT FROM SMF TO LAX; WE WERE CLRED FIM 6 ...,NaN
...,...,...,...,...,...
204994,665689,Human Factors; Aircraft; Company Policy; Envir...,Human Factors,#7 TIRE TIE BOLT MISSING.,Main Gear Wheel
204995,665690,Environment - Non Weather Related; Human Facto...,Human Factors,WHEEL #6 BROKEN BOLT.,Main Gear Wheel
204996,665691,Aircraft; Chart Or Publication; Company Policy...,Human Factors,#4 MAIN WHEEL ASSEMBLY HAS 2 TIE BOLTS SHEARED...,Main Gear Wheel
204997,665692,Human Factors; Aircraft; Logbook Entry; Chart ...,Human Factors,FOUND 1 TIE BOLT NUT MISSING FROM WHEEL ASSEMB...,Main Gear Wheel


# Cleaning Up Target Variable

## We want the final data frame to have four categories for our target variable "human factors", "procedures", and "aircraft" so that we can compare to SafeAeroBERT.

In [ ]:
big_df["Assessments.1"].value_counts()

# include missing values in the tally
big_df["Assessments.1"].value_counts(dropna=False)

,count
Assessments.1,
Human Factors,103477
Aircraft,44350
Ambiguous,12059
Procedure,8932
NaN,7572
Company Policy,7296
Weather,6171
Airport,4201
ATC Equipment / Nav Facility / Buildings,2687


## If an entry contains the string one of "Aircraft", "Procedure", "Weather", "Human Factors" it will be replaced with that given word. If multiple are in an entry whichever one shows up first will replace.                                 
Example: "Weather; Human Factors" -> "Weather"

In [ ]:
import pandas as pd
import numpy as np

# -- parameters ---------------------------------------------------------------
TARGET_COL   = "Assessments.1"                 # change if your spelling differs
CATEGORIES   = ["Aircraft", "Procedure", "Weather", "Human Factors"]

# -- helper -------------------------------------------------------------------
def normalize_assessment(val: str) -> str:
    """
    Replace *val* with the first category token that appears inside it.
    If none appear, or the cell is NaN/empty, return the original value.
    """
    if pd.isna(val) or val == "":
        return val

    text = str(val).lower()                    # case-insensitive search
    matches = {cat: text.find(cat.lower())     # index of each cat, -1 if absent
               for cat in CATEGORIES}
    # keep only found categories (index >= 0)
    found = {cat: idx for cat, idx in matches.items() if idx >= 0}

    if found:                                  # at least one category present
        # pick the one whose index is smallest → appears earliest in the string
        return min(found, key=found.get)

    return val                                 # no category token → leave untouched

# -- apply --------------------------------------------------------------------
big_df[TARGET_COL] = big_df[TARGET_COL].apply(normalize_assessment)

In [ ]:
# ‼️ make sure the column name is spelled exactly as in the DataFrame
# If you used "Assesments.1" earlier, change it here too.
big_df["Assessments.1"].value_counts()         # descending order, NaNs omitted

# include missing values in the tally
big_df["Assessments.1"].value_counts(dropna=False)

,count
Assessments.1,
Human Factors,103542
Aircraft,44411
Ambiguous,12059
Procedure,8934
Weather,8854
NaN,7572
Company Policy,7296
Airport,4201
ATC Equipment / Nav Facility / Buildings,2687


## We will now map the remaining unique values in our target variable column

In [ ]:
import pandas as pd, numpy as np

# -------------------------------------------------------------------
# 1)  Token → Category dictionary  (extend / tweak as you like)
# -------------------------------------------------------------------
TOKEN_TO_CATEGORY = {
    # ---- WEATHER ---------------------------------------------------
    "weather":                   "Weather",

    # ---- HUMAN FACTORS --------------------------------------------
    "human factors":             "Human Factors",
    "human":                     "Human Factors",

    # ---- AIRCRAFT --------------------------------------------------
    "aircraft":                  "Aircraft",
    "mel":                       "Aircraft",
    "m.e.l":                     "Aircraft",
    "equipment":                 "Aircraft",
    "tooling":                   "Aircraft",
    "atc equipment":             "Aircraft",
    "nav facility":              "Aircraft",
    "software":                  "Aircraft",
    "buildings":                 "Aircraft",
    "incorrect / not installed / unavailable part": "Aircraft",

    # ---- PROCEDURE -------------------------------------------------
    "staffing":                  "Procedure",
    "mel":                       "Procedure",
    "procedure":                 "Procedure",
    "chart":                     "Procedure",
    "publication":               "Procedure",
    "company policy":            "Procedure",
    "manual":                    "Procedure",
    "manuals":                   "Procedure",
    "logbook":                   "Procedure",
    "logbook entry":             "Procedure",
    "airport":                   "Procedure",
    "airspace":                  "Procedure",
    "automation":                "Procedure",
}

# -------------------------------------------------------------------
# 2)  Mapper that uses the dictionary
# -------------------------------------------------------------------
def map_assessment(cell: str) -> str | float:
    """
    Replace *cell* with the category attached to the first matching token
    from TOKEN_TO_CATEGORY.  Return NaN if nothing matches.
    """
    if pd.isna(cell) or cell == "":
        return np.nan

    text = str(cell).lower()            # case-insensitive scan
    earliest_category = None
    earliest_index    = len(text) + 1   # bigger than any real index

    for token, category in TOKEN_TO_CATEGORY.items():
        idx = text.find(token)
        if idx != -1 and idx < earliest_index:
            earliest_index    = idx
            earliest_category = category

    return earliest_category if earliest_category else np.nan

# -------------------------------------------------------------------
# 3)  Apply to your DataFrame
# -------------------------------------------------------------------
big_df["Assessments.1_norm"] = big_df["Assessments.1"].apply(map_assessment)

# -------------------------------------------------------------------
# 4)  Quick sanity-check
# -------------------------------------------------------------------
big_df["Assessments.1_norm"].value_counts(dropna=True)

,count
Assessments.1_norm,
Human Factors,103542
Aircraft,47888
Procedure,25084
Weather,8854


# Renaming columns so that they have a more intuitive name and save

In [ ]:
# ---------------------------------------------------------------
# 1) Drop the old Assessments.1 (we've normalised it already)
# ---------------------------------------------------------------
big_df = big_df.drop(columns=["Assessments.1"])

# ---------------------------------------------------------------
# 2) Drop rows where Assessments.1_norm is NaN
# ---------------------------------------------------------------
big_df = big_df.dropna(subset=["Assessments.1_norm"]).reset_index(drop=True)

# ---------------------------------------------------------------
# 3) Rename columns so they're consistent and typo-free
# ---------------------------------------------------------------
rename_map = {
    "ACN": "Aircraft Number",
    "Assessments":    "Contributing Factors",      # fix the missing "s"
    "Assessments.1_norm":  "Primary Factors", # fix the missing "s"
    "Report 1": "Reports"
    # feel free to add any others, e.g. "Report 1": "Report"
}
big_df = big_df.rename(columns=rename_map)

# ---------------------------------------------------------------
# 4) Save the dataframe as a csv (LOCAL PATH - not Google Drive)
# ---------------------------------------------------------------
out_file = Path("./data/final/NASA_ASRS.csv")
out_file.parent.mkdir(parents=True, exist_ok=True)  # Ensure the folder exists
big_df.to_csv(out_file, index=False)

print(f"✓ Saved final file to: {out_file.absolute()}")

big_df

,Aircraft Number,Contributing Factors,Reports,Component,Primary Factors
0,177042,NaN,CABIN PRESSURE LOSS AT 35000'; RAPID DSNT MADE...,NaN,Aircraft
1,177186,Airport,JUST AFTER T/D ON RWY 27 AT Y31; A DEER RAN ON...,NaN,Procedure
2,177189,NaN,I DEPARTED SOUTH LAKE TAHOE ARPT ON A DIRECT F...,NaN,Human Factors
3,177197,NaN,I TOOK OFF OUT OF VICTORIA ARPT AND WAS IN IMC...,NaN,Human Factors
4,177205,NaN,ON A FLT FROM SMF TO LAX; WE WERE CLRED FIM 6 ...,NaN,Human Factors
...,...,...,...,...,...
185363,665689,Human Factors; Aircraft; Company Policy; Envir...,#7 TIRE TIE BOLT MISSING.,Main Gear Wheel,Human Factors
185364,665690,Environment - Non Weather Related; Human Facto...,WHEEL #6 BROKEN BOLT.,Main Gear Wheel,Human Factors
185365,665691,Aircraft; Chart Or Publication; Company Policy...,#4 MAIN WHEEL ASSEMBLY HAS 2 TIE BOLTS SHEARED...,Main Gear Wheel,Human Factors
185366,665692,Human Factors; Aircraft; Logbook Entry; Chart ...,FOUND 1 TIE BOLT NUT MISSING FROM WHEEL ASSEMB...,Main Gear Wheel,Human Factors
